## Sudoku Solver using OpenCV

In [1]:
import cv2
import numpy as np

In [2]:
# Load the sudoku puzzle images
puzzle1 = cv2.imread('Puzzles/puzzle1.png')
puzzle2 = cv2.imread('Puzzles/puzzle2.png')

# Display the images
cv2.imshow('Puzzle 1', puzzle1)
cv2.imshow('Puzzle 2', puzzle2)
cv2.waitKey(5)
cv2.destroyAllWindows()

## Process the images

In [4]:
def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (7, 7), 3)
    
    # Apply adaptive thresholding to get binary image
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY_INV, 11, 2)
    
    return thresh

def find_puzzle(image):
    # Preprocess the image
    processed = preprocess_image(image)
    
    # Find contours
    contours, _ = cv2.findContours(processed.copy(), cv2.RETR_EXTERNAL,
                                  cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours by area in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    puzzle_contour = None
    
    # Loop through contours to find the puzzle grid
    for contour in contours:
        # Approximate the contour
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        
        # If we have found a rectangle with 4 corners, we've found our puzzle
        if len(approx) == 4:
            puzzle_contour = approx
            break
            
    return puzzle_contour

# Process both puzzles
puzzle1_processed = preprocess_image(puzzle1)
puzzle2_processed = preprocess_image(puzzle2)

# Find puzzle contours
puzzle1_contour = find_puzzle(puzzle1)
puzzle2_contour = find_puzzle(puzzle2)

# Draw the contours on the original images
if puzzle1_contour is not None:
    cv2.drawContours(puzzle1, [puzzle1_contour], -1, (0, 255, 0), 2)
if puzzle2_contour is not None:
    cv2.drawContours(puzzle2, [puzzle2_contour], -1, (0, 255, 0), 2)

# Display results
cv2.imshow('Processed Puzzle 1', puzzle1_processed)
cv2.imshow('Processed Puzzle 2', puzzle2_processed)
cv2.imshow('Detected Grid 1', puzzle1)
cv2.imshow('Detected Grid 2', puzzle2)
cv2.waitKey(5)
cv2.destroyAllWindows()


## Download digit templates

In [5]:
import os
import numpy as np
from tensorflow.keras.datasets import mnist
from PIL import Image

# Load the MNIST dataset
(train_images, train_labels), (_, _) = mnist.load_data()

# Specify the output directory
output_dir = 'mnist_digits'
os.makedirs(output_dir, exist_ok=True)

# Define the digits to save
digits_to_save = range(1, 10)

# Initialize a dictionary to track saved digits
saved_digits = {digit: False for digit in digits_to_save}

# Iterate over the dataset to find and save one image per digit
for image, label in zip(train_images, train_labels):
    if label in digits_to_save and not saved_digits[label]:
        # Convert the image to a PIL Image object
        pil_image = Image.fromarray(image)
        # Save the image as a PNG file
        pil_image.save(os.path.join(output_dir, f'digit_{label}.png'))
        # Mark this digit as saved
        saved_digits[label] = True
        print(f'Saved digit {label} as digit_{label}.png')
    # Break the loop if all digits have been saved
    if all(saved_digits.values()):
        break

print('All specified digit images have been saved.')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Saved digit 5 as digit_5.png
Saved digit 4 as digit_4.png
Saved digit 1 as digit_1.png
Saved digit 9 as digit_9.png
Saved digit 2 as digit_2.png
Saved digit 3 as digit_3.png
Saved digit 6 as digit_6.png
Saved digit 7 as digit_7.png
Saved digit 8 as digit_8.png
All specified digit images have been saved.


## Solve the puzzle

In [7]:
def extract_digits(processed_image, contour):
    # Get perspective transform
    pts = contour.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    
    # Order points: top-left, top-right, bottom-right, bottom-left
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    # Get width and height of the grid
    widthA = np.sqrt(((rect[2][0] - rect[3][0]) ** 2) + ((rect[2][1] - rect[3][1]) ** 2))
    widthB = np.sqrt(((rect[1][0] - rect[0][0]) ** 2) + ((rect[1][1] - rect[0][1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    
    heightA = np.sqrt(((rect[1][0] - rect[2][0]) ** 2) + ((rect[1][1] - rect[2][1]) ** 2))
    heightB = np.sqrt(((rect[0][0] - rect[3][0]) ** 2) + ((rect[0][1] - rect[3][1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    
    # Apply perspective transform
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(processed_image, M, (maxWidth, maxHeight))
    
    # Create 9x9 grid
    cell_height = maxHeight // 9
    cell_width = maxWidth // 9
    grid = np.zeros((9, 9), dtype=int)
    
    # Extract each cell
    for i in range(9):
        for j in range(9):
            cell = warped[i*cell_height:(i+1)*cell_height, j*cell_width:(j+1)*cell_width]
            
            # Add padding to cell
            pad = 5
            cell = cell[pad:-pad, pad:-pad]
            
            # If cell is mostly white (empty), set to 0
            if np.mean(cell) > 200:
                grid[i][j] = 0
                continue
            
            # Use template matching to recognize digits
            best_match = 0
            best_score = float('-inf')
            
            # Resize cell to standard size
            cell = cv2.resize(cell, (28, 28))
            
            # Simple thresholding to make digit recognition more robust
            _, cell = cv2.threshold(cell, 127, 255, cv2.THRESH_BINARY)
            
            # Compare with each digit template (1-9)
            for digit in range(1, 10):
                # Create a basic template for each digit
                # This is a very simple approach - you might want to use better templates
                template = cv2.imread(f'digit_templates/digit_{digit}.png', 0)
                if template is None:
                    continue
                    
                template = cv2.resize(template, (28, 28))
                result = cv2.matchTemplate(cell, template, cv2.TM_CCOEFF_NORMED)
                score = np.max(result)
                
                if score > best_score:
                    best_score = score
                    best_match = digit
            
            # If confidence is too low, assume empty cell
            if best_score < 0.5:
                grid[i][j] = 0
            else:
                grid[i][j] = best_match
                
    return grid

def solve_puzzle(image, contour):
    # First extract the grid
    grid = extract_digits(image, contour)
    
    # Solve the Sudoku
    if solve_sudoku(grid):
        return grid
    else:
        return None

# Add the solving functions from previous response here
def is_valid(board, num, pos):
    # Check row
    for x in range(len(board[0])):
        if board[pos[0]][x] == num and pos[1] != x:
            return False
            
    # Check column
    for x in range(len(board)):
        if board[x][pos[1]] == num and pos[0] != x:
            return False
    
    # Check 3x3 box
    box_x = pos[1] // 3
    box_y = pos[0] // 3
    for i in range(box_y * 3, box_y * 3 + 3):
        for j in range(box_x * 3, box_x * 3 + 3):
            if board[i][j] == num and (i, j) != pos:
                return False
    
    return True

def find_empty(board):
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0:
                return (i, j)
    return None

def solve_sudoku(board):
    empty = find_empty(board)
    if not empty:
        return True
    
    row, col = empty
    
    for num in range(1, 10):
        if is_valid(board, num, (row, col)):
            board[row][col] = num
            
            if solve_sudoku(board):
                return True
            
            board[row][col] = 0
    
    return False

# Solve both puzzles
solution1 = solve_puzzle(puzzle1_processed, puzzle1_contour)
solution2 = solve_puzzle(puzzle2_processed, puzzle2_contour)

# Display results
if solution1 is not None:
    print("Solution for Puzzle 1:")
    print(np.array(solution1))
else:
    print("Could not solve Puzzle 1")

if solution2 is not None:
    print("\nSolution for Puzzle 2:")
    print(np.array(solution2))
else:
    print("Could not solve Puzzle 2")

Solution for Puzzle 1:
[[1 2 3 4 5 6 7 8 9]
 [4 5 6 7 8 9 1 2 3]
 [7 8 9 1 2 3 4 5 6]
 [2 1 4 3 6 5 8 9 7]
 [3 6 5 8 9 7 2 1 4]
 [8 9 7 2 1 4 3 6 5]
 [5 3 1 6 4 2 9 7 8]
 [6 4 2 9 7 8 5 3 1]
 [9 7 8 5 3 1 6 4 2]]

Solution for Puzzle 2:
[[1 2 3 4 5 6 7 8 9]
 [4 5 6 7 8 9 1 2 3]
 [7 8 9 1 2 3 4 5 6]
 [2 1 4 3 6 5 8 9 7]
 [3 6 5 8 9 7 2 1 4]
 [8 9 7 2 1 4 3 6 5]
 [5 3 1 6 4 2 9 7 8]
 [6 4 2 9 7 8 5 3 1]
 [9 7 8 5 3 1 6 4 2]]


[ WARN:0@328.093] global loadsave.cpp:241 findDecoder imread_('digit_templates/1.png'): can't open/read file: check file path/integrity
[ WARN:0@328.093] global loadsave.cpp:241 findDecoder imread_('digit_templates/2.png'): can't open/read file: check file path/integrity
[ WARN:0@328.093] global loadsave.cpp:241 findDecoder imread_('digit_templates/3.png'): can't open/read file: check file path/integrity
[ WARN:0@328.093] global loadsave.cpp:241 findDecoder imread_('digit_templates/4.png'): can't open/read file: check file path/integrity
[ WARN:0@328.093] global loadsave.cpp:241 findDecoder imread_('digit_templates/5.png'): can't open/read file: check file path/integrity
[ WARN:0@328.093] global loadsave.cpp:241 findDecoder imread_('digit_templates/6.png'): can't open/read file: check file path/integrity
[ WARN:0@328.093] global loadsave.cpp:241 findDecoder imread_('digit_templates/7.png'): can't open/read file: check file path/integrity
[ WARN:0@328.093] global loadsave.cpp:241 findDe